# iNaturalist status updates by state

Using the file produced in the collate-status-taxa.ipynb: `inat-aust-status-taxa.csv`, generate lists to update iNaturalist statuses

**Next steps:**
State by state establish the changes that need to be made:
    a. new - any new species that appear in the state lists but do not have a status in inaturalist (new template)
    b. updates - any changes to information which was added by us previously (user_id = 708886) (update template, action='UPDATE')
    c. removals - any statuses which were added by us previously (user_id = 708886) list which are incorrect (update template, action='REMOVE')
    d. flags - are there any statuses by other users that need to be flagged?

## Prep - common to all states
1. Read in the inaturalist statuses & filter out this state
2. Read in the inaturalist taxa list
3. Read in the state sensitive list
4. Attempt to match the state statuses to an IUCN equivalent


### 1. iNaturalist statuses

In [82]:
import sys
import os
projectdir = "/Users/oco115/PycharmProjects/authoritative-lists/" # basedir for this gh project
sys.path.append(os.path.abspath(projectdir + "source-code/includes"))
import list_functions as lf

import pandas as pd
# projectdir = "/Users/new330/IdeaProjects/authoritative-lists/" # basedir for this gh project
sourcedir = projectdir + "source-data/inaturalist-statuses/"
listdir = projectdir + "current-lists/"


# read in the statuses
taxastatus = pd.read_csv(sourcedir + "inat-aust-status-taxa.csv", encoding='UTF-8',na_filter=False,dtype=str) ## Read inaturalist conservation statuses file
taxastatus.head(3)

,id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
0,166449,38493,1138587,7830,,Flora and Fauna Guarantee Act 1988,CR,,,obscured,...,Eulamprus,kosciuskoi,,2021-03-01T10:35:01Z,Eulamprus kosciuskoi,species,http://reptile-database.reptarium.cz/search.ph...,,,
1,234788,918383,702203,9994,,Atlas of Living Australia,NT,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,
2,234789,918383,702203,7308,,Atlas of Living Australia,LC,https://bie.ala.org.au/species/https://id.biod...,,,...,Chiloschista,phyllorhiza,,2022-01-08T03:30:36Z,Chiloschista phyllorhiza,species,http://www.catalogueoflife.org/annual-checklis...,,,


In [83]:
def filter_state_statuses(stateregex: str, urlregex: str):
    authoritydf = taxastatus['authority'].drop_duplicates().sort_values()
    authoritydf = authoritydf[pd.Series(authoritydf).str.contains(stateregex)]
    urldf = taxastatus['url'].drop_duplicates().sort_values()
    urldf = urldf[pd.Series(urldf).str.contains(urlregex)]
    placedisplaydf = taxastatus['place_display_name'].drop_duplicates().sort_values()
    placedisplaydf = placedisplaydf[pd.Series(placedisplaydf).str.contains(stateregex)]
    placedf = taxastatus['place_name'].drop_duplicates().sort_values()
    placedf = placedf[pd.Series(placedf).str.contains(stateregex)]
    # concat all and remove duplicates
    statedf = pd.concat([taxastatus.apply(lambda row: row[taxastatus['place_display_name'].isin(placedisplaydf)]),
                         taxastatus.apply(lambda row: row[taxastatus['place_name'].isin(placedf)]),
                         taxastatus.apply(lambda row: row[taxastatus['url'].isin(urldf)]),
                         taxastatus.apply(
                             lambda row: row[taxastatus['authority'].isin(authoritydf)])]).drop_duplicates()
    return statedf.sort_values(['taxon_id', 'user_id'])
inatstatuses = filter_state_statuses("Tasmania|TAS", ".tas.gov")
inatstatuses.rename(columns={'id':'status_id','id_y':'taxon_id_y'},inplace=True)
inatstatuses

,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
212,170159,1007040,702203,6829,,Threatened Species Protection Act 1995,EN,https://www.threatenedspecieslink.tas.gov.au/P...,,,...,Dianella,amoena,,2021-07-27T17:12:17Z,Dianella amoena,species,http://www.catalogueoflife.org/annual-checklis...,,,
2558,160921,1038747,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Ranunculus...,,obscured,...,Ranunculus,diminutus,,2022-06-13T17:38:59Z,Ranunculus diminutus,species,,,,
2547,160910,1038906,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Myosurus-a...,,obscured,...,Myosurus,australis,,2020-08-19T03:33:32Z,Myosurus australis,species,,,,
933,160738,1038964,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Boronia%20...,,obscured,...,Boronia,hemichiton,,2020-08-17T05:51:50Z,Boronia hemichiton,species,,,,
2557,160920,1038976,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Pimelea-ax...,,obscured,...,Pimelea,axiflora,axiflora,2020-12-02T04:00:26Z,Pimelea axiflora axiflora,subspecies,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2586,160759,958863,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Corunastyl...,,,...,Genoplesium,nudiscapum,,2022-10-26T09:34:12Z,Genoplesium nudiscapum,species,https://eol.org/pages/1099120,,,
2594,161103,960004,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Thelymitra...,,obscured,...,Thelymitra,bracteata,,2022-07-14T00:09:44Z,Thelymitra bracteata,species,http://www.catalogueoflife.org/annual-checklis...,,,
3380,268029,993886,1116633,6829,,TASMANIAN THREATENED SPECIES LISTING STATEMENT,EN,https://nre.tas.gov.au/Documents/Prasophyllum%...,1. total population estimated to number fewer ...,obscured,...,Prasophyllum,abblittiorum,,2020-02-18T23:45:10Z,Prasophyllum abblittiorum,species,https://eol.org/pages/51504614,,,
813,160748,993902,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Caladenia-...,,,...,Caladenia,dienema,,2020-12-15T00:50:01Z,Caladenia dienema,species,http://www.catalogueoflife.org/annual-checklis...,,,


### 2. iNaturalist taxonomy

In [84]:
# Output files contain these fields
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# so we need to match species from the state lists to the inat taxa to get the taxon_id

import zipfile
url = "https://www.inaturalist.org/taxa/inaturalist-taxonomy.dwca.zip"
filename = url.split("/")[-1]

z=zipfile.ZipFile(sourcedir + filename)

with z.open('taxa.csv') as from_archive:
    inattaxa = pd.read_csv(from_archive,dtype=str)
z.close()
inattaxa.head(3)


,id,taxonID,identifier,parentNameUsageID,kingdom,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references
0,1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/1,https://www.inaturalist.org/taxa/48460,Animalia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-02T06:05:44Z,Animalia,kingdom,http://www.catalogueoflife.org/annual-checklis...
1,2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/2,https://www.inaturalist.org/taxa/1,Animalia,Chordata,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-23T00:40:18Z,Chordata,phylum,http://www.catalogueoflife.org/annual-checklis...
2,3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/3,https://www.inaturalist.org/taxa/355675,Animalia,Chordata,Aves,NaN,NaN,NaN,NaN,NaN,2022-12-27T07:33:16Z,Aves,class,http://www.catalogueoflife.org/annual-checklis...


### 3. State lists
Sensitive list: `geoprivacy` = `obscured`
overrides

In [85]:
sensitivelist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr491?max=10000&includeKVP=true")
sensitivelist['scientificName'] = sensitivelist['scientificName'].str.replace('subsp. ', '', regex=False)
sensitivelist['bionet_geoprivacy'] = 'obscured'
# sensitivelist = sensitivelist.rename(columns={'taxonID':'wildnetTaxonID'})
sensitivelist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,bionet_geoprivacy
0,24386,Lomatia tasmanica,King's Lomatia,Lomatia tasmanica,https://id.biodiversity.org.au/node/apni/2903191,dr491,[],obscured


In [86]:
conservationlist = lf.download_ala_list("https://lists.ala.org.au/ws/speciesListItems/dr654?max=10000&includeKVP=true")
conservationlist = lf.kvp_to_columns(conservationlist)
conservationlist['scientificName'] = conservationlist['scientificName'].str.replace('subsp. ', '', regex=False)
conservationlist['bionet_geoprivacy'] = 'open'
conservationlist

,id,name,commonName,scientificName,lsid,dataResourceUid,kvpValues,scientificNameAuthorship,vernacular name,family,class,sourceStatus,status,sourceKingdom,kingdom,speciesGroup,sourceOrigin,higherGeography,E P B C A_status,bionet_geoprivacy
0,4033150,Costora iena,Caddis Fly (great Lakes),Costora iena,https://biodiversity.org.au/afd/taxa/7de808fb-...,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",Mosely,Caddis Fly (Great Lakes),Trichoptera,Invertebrates,x,Presumed Extinct,Fauna,Animalia,invertebrate,end,endemic Tasmania,NaN,open
0,4033128,Hadronyche pulvinator,Cascade Funnel-web Spider,Hadronyche pulvinator,https://biodiversity.org.au/afd/taxa/2acd0df1-...,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",Hickman,Cascade Funnel-web Spider,Araneae,Invertebrates,x,Presumed Extinct,Fauna,Animalia,invertebrate,end,endemic Tasmania,NaN,open
0,4033378,Hypolimnus pedderensis,Lake Pedder Earthworm,Hypolimnus pedderensis,https://biodiversity.org.au/afd/taxa/7db09aa3-...,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",(Jamieson),Lake Pedder Earthworm,Haplotaxida,Invertebrates,x,Presumed Extinct,Fauna,Animalia,invertebrate,end,endemic Tasmania,EX,open
0,4033234,Thylacinus cynocephalus,Thylacine,Thylacinus cynocephalus,https://biodiversity.org.au/afd/taxa/97764bed-...,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",Harris,Thylacine,Mammals,Vertebrates,x,Presumed Extinct,Fauna,Animalia,vertebrate,end,endemic Tasmania,EX,open
0,4033416,Gallirallus philippensis macquariensis,None,Gallirallus philippensis macquariensis,NZOR-6-101588,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",Hutton,Macquarie Island Rail,Birds,Vertebrates,x,Presumed Extinct,Fauna,Animalia,vertebrate,eMI,endemic Macquarie Island,EX,open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4033369,Erioderma sorediatum,None,Erioderma sorediatum,ALA_DR654_679,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",D.J.Galloway & P.M.Jørg.,NaN,Pannariaceae,Lichen,e,Endangered,Flora,Plantae,non-vascular,NaN,NaN,NaN,open
0,4033419,Xanthoparmelia amphixantha,None,Xanthoparmelia amphixantha,ALA_DR654_680,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",(Müll.Arg.) Hale,NaN,Parmeliaceae,Lichen,e,Endangered,Flora,Plantae,non-vascular,NaN,NaN,NaN,open
0,4033291,Xanthoparmelia molliuscula,None,Xanthoparmelia molliuscula,ALA_DR654_681,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...",(Ach.) Hale,NaN,Parmeliaceae,Lichen,e,Endangered,Flora,Plantae,non-vascular,NaN,NaN,NaN,open
0,4033236,Xanthoparmelia subloxodella,None,Xanthoparmelia subloxodella,ALA_DR654_682,dr654,"[{'key': 'scientificNameAuthorship', 'value': ...","(Elix & Kantvilas) O.Blanco, A.Crespo, Elix, D...",NaN,Parmeliaceae,Lichen,e,Endangered,Flora,Plantae,non-vascular,NaN,NaN,NaN,open


In [87]:
# join them in a way that works for inat (eg sensitive list, geoprivacy = 'obscured'
statelist = pd.concat([sensitivelist[['scientificName','bionet_geoprivacy', 'lsid']],
                    conservationlist[['scientificName','status','bionet_geoprivacy', 'lsid']]]).drop_duplicates()
statelist

,scientificName,bionet_geoprivacy,lsid,status
0,Lomatia tasmanica,obscured,https://id.biodiversity.org.au/node/apni/2903191,NaN
0,Costora iena,open,https://biodiversity.org.au/afd/taxa/7de808fb-...,Presumed Extinct
0,Hadronyche pulvinator,open,https://biodiversity.org.au/afd/taxa/2acd0df1-...,Presumed Extinct
0,Hypolimnus pedderensis,open,https://biodiversity.org.au/afd/taxa/7db09aa3-...,Presumed Extinct
0,Thylacinus cynocephalus,open,https://biodiversity.org.au/afd/taxa/97764bed-...,Presumed Extinct
...,...,...,...,...
0,Erioderma sorediatum,open,ALA_DR654_679,Endangered
0,Xanthoparmelia amphixantha,open,ALA_DR654_680,Endangered
0,Xanthoparmelia molliuscula,open,ALA_DR654_681,Endangered
0,Xanthoparmelia subloxodella,open,ALA_DR654_682,Endangered


In [88]:
# check for duplicates with conflicting information
statelist.groupby('scientificName').filter(lambda x: len(x) > 1)#.sort('size',ascending=False)


,scientificName,bionet_geoprivacy,lsid,status
0,Lomatia tasmanica,obscured,https://id.biodiversity.org.au/node/apni/2903191,NaN
0,Lomatia tasmanica,open,https://id.biodiversity.org.au/node/apni/2903191,Endangered


### 4. Equivalent IUCN statuses

In [89]:
# iucn_statuses = {'Not Evaluated', 'Data Deficient', 'Least Concern', 'Near Threatened', 'Vulnerable', 'Endangered', 'Critically Endangered', 'Extinct in the Wild', 'Extinct'}
statelist.groupby(['status'])['status'].count()

status
Endangered          222
Presumed Extinct     28
Rare                310
Vulnerable          122
Name: status, dtype: int64

In [90]:
iucnStatusMappings = {
    'Presumed Extinct':'Extinct',
    'Rare':'Vulnerable'
}

### 5. Determine best place ID to use

In [91]:
inatstatuses.groupby(['place_id','place_name','place_display_name'])['place_id'].count()
# looks like 6825

place_id  place_name          place_display_name    
6744      Australia           Australia                  2
6829      Tasmania            Tasmania, AU              77
9994      Northern Territory  Northern Territory, AU     6
Name: place_id, dtype: int64

## Merge iNaturalist statuses with State sensitive list on scientificName

1. Match - updates, even if the statuses are the same we'll update the links and values anyway
2. No match - statuses to be added (additions)
   1.1 No match and no taxnomy - search for synonyms
   1.2 No match
3. Merge the other direction to see if there are deletes?


In [92]:
# join to see which lists already have a status in inaturalist based on scientificName
mergedstatuses = statelist[['scientificName','status','bionet_geoprivacy', 'lsid']].merge(inatstatuses[['status_id','scientificName','taxon_id','user_id','description','iucn','authority','status','geoprivacy','place_id','place_display_name']],how="left",left_on='scientificName',right_on='scientificName',suffixes=(None,'_inat')).sort_values(['scientificName'])
mergedstatuses


,scientificName,status,bionet_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,status_inat,geoprivacy,place_id,place_display_name
216,Acacia axillaris,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2911902,160726,775192,717122,,30,Environment Protection and Biodiversity Conser...,VU,obscured,6744,Australia
300,Acacia pataczekii,Rare,open,https://id.biodiversity.org.au/node/apni/2913609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355,Acacia siculiformis,Rare,open,https://id.biodiversity.org.au/node/apni/2888960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,Acacia ulicifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2907350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,Acacia uncifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2916270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,Xerochrysum bicolor,Rare,open,https://id.biodiversity.org.au/node/apni/2902500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287,Xerochrysum palustre,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2890165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,Zearaja maugeana,Endangered,open,https://biodiversity.org.au/afd/taxa/0d9d194f-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
521,Zieria littoralis,Rare,open,https://id.biodiversity.org.au/node/apni/2888662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
# prepare the export fields, common to New template and Update template
# new statuses
# Taxon_Name,Status,Authority,IUCN_equivalent,Description,iNaturalist_Place_ID,url,Taxon_Geoprivacy,Username,taxon_id
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username
mergedstatuses['new_authority'] = "Threatened Species Protection Act 1995"
mergedstatuses['new_description'] = "Listed as Threatened - refer to https://www.threatenedspecieslink.tas.gov.au"
mergedstatuses['new_url'] = mergedstatuses['lsid']
mergedstatuses['new_geoprivacy'] = "obscured"
mergedstatuses['new_place_id'] = '6829'  # Tasmania
mergedstatuses['new_username'] = 'peggydnew'
mergedstatuses['new_iucn_equivalent'] = mergedstatuses['status'].str.lower().str.strip().map(iucnStatusMappings).fillna('Vulnerable') # map to dictionary
mergedstatuses['new_status'] = mergedstatuses['status'].fillna('Threatened')
mergedstatuses

,scientificName,status,bionet_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,place_id,place_display_name,new_authority,new_description,new_url,new_geoprivacy,new_place_id,new_username,new_iucn_equivalent,new_status
216,Acacia axillaris,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2911902,160726,775192,717122,,30,Environment Protection and Biodiversity Conser...,...,6744,Australia,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2911902,obscured,6829,peggydnew,Vulnerable,Vulnerable
300,Acacia pataczekii,Rare,open,https://id.biodiversity.org.au/node/apni/2913609,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2913609,obscured,6829,peggydnew,Vulnerable,Rare
355,Acacia siculiformis,Rare,open,https://id.biodiversity.org.au/node/apni/2888960,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2888960,obscured,6829,peggydnew,Vulnerable,Rare
356,Acacia ulicifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2907350,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2907350,obscured,6829,peggydnew,Vulnerable,Rare
357,Acacia uncifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2916270,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2916270,obscured,6829,peggydnew,Vulnerable,Rare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,Xerochrysum bicolor,Rare,open,https://id.biodiversity.org.au/node/apni/2902500,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2902500,obscured,6829,peggydnew,Vulnerable,Rare
287,Xerochrysum palustre,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2890165,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2890165,obscured,6829,peggydnew,Vulnerable,Vulnerable
173,Zearaja maugeana,Endangered,open,https://biodiversity.org.au/afd/taxa/0d9d194f-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://biodiversity.org.au/afd/taxa/0d9d194f-...,obscured,6829,peggydnew,Vulnerable,Endangered
521,Zieria littoralis,Rare,open,https://id.biodiversity.org.au/node/apni/2888662,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Threatened Species Protection Act 1995,Listed as Threatened - refer to https://www.th...,https://id.biodiversity.org.au/node/apni/2888662,obscured,6829,peggydnew,Vulnerable,Rare


## Updates

In [94]:
# updates
# action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
updates = pd.DataFrame(mergedstatuses[mergedstatuses['status_id'].notnull()])
updates.sort_values('scientificName')
updates['action'] = 'UPDATE'
#updates.loc[:,'action'] = 'UPDATE'
updates = updates[['action','scientificName','status_id','taxon_id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
updates.columns = updates.columns.str.replace("new_", "", regex=True)
updates = updates.rename(columns={'scientificName':'taxon_name',
                                  'status_id':'id'})
updates

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
216,UPDATE,Acacia axillaris,160726,775192,Vulnerable,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2911902,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
603,UPDATE,Alternanthera denticulata,160727,369665,Endangered,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/taxon/apni/5126...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
604,UPDATE,Amphibromus macrorhinus,160728,1121410,Endangered,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2916650,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
243,UPDATE,Anogramma leptophylla,162081,180646,Vulnerable,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/taxon/apni/5129...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
360,UPDATE,Aphelia gracilis,163877,1143399,Rare,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2899707,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
...,...,...,...,...,...,...,...,...,...,...,...,...
670,UPDATE,Thelymitra malvina,161104,406524,Endangered,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/taxon/apni/5141...,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
673,UPDATE,Veronica notabilis,161105,791936,Endangered,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2890187,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
674,UPDATE,Viminaria juncea,161106,353911,Endangered,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2904505,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
676,UPDATE,Wurmbea latifolia vanessae,161107,790432,Endangered,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2897412,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...


## No status in iNaturalist via straight scientificName match
The ACT records that didn't match up to a status in iNaturalist

In [95]:
# to add: those that have no inaturalist status - 532!!
noinatstatus = mergedstatuses[mergedstatuses['status_id'].isnull()]
# try to match the taxon name to something in inaturalist
noinatstatus = noinatstatus.merge(inattaxa, how="left", left_on="scientificName",right_on="scientificName")
noinatstatus

,scientificName,status,bionet_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,Acacia pataczekii,Rare,open,https://id.biodiversity.org.au/node/apni/2913609,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,pataczekii,NaN,2022-04-06T23:48:02Z,species,http://www.catalogueoflife.org/annual-checklis...
1,Acacia siculiformis,Rare,open,https://id.biodiversity.org.au/node/apni/2888960,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,siculiformis,NaN,2022-04-07T00:40:40Z,species,http://www.catalogueoflife.org/annual-checklis...
2,Acacia ulicifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2907350,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,ulicifolia,NaN,2022-06-18T06:33:18Z,species,http://www.catalogueoflife.org/annual-checklis...
3,Acacia uncifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2916270,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,uncifolia,NaN,2022-06-10T10:52:23Z,species,http://eol.org/pages/26965172
4,Acanthiza pusilla archibaldi,Endangered,open,ALA_DR654_159,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Passeriformes,Acanthizidae,Acanthiza,pusilla,archibaldi,2018-12-19T06:16:34Z,subspecies,http://www.birds.cornell.edu/clementschecklist...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,Xanthorrhoea bracteata,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2911139,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Asphodelaceae,Xanthorrhoea,bracteata,NaN,2022-08-25T07:03:45Z,species,http://powo.science.kew.org/
605,Xerochrysum bicolor,Rare,open,https://id.biodiversity.org.au/node/apni/2902500,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Asterales,Asteraceae,Xerochrysum,bicolor,NaN,2020-02-19T08:20:14Z,species,http://www.catalogueoflife.org/annual-checklis...
606,Xerochrysum palustre,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2890165,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Asterales,Asteraceae,Xerochrysum,palustre,NaN,2022-06-08T17:48:57Z,species,http://eol.org/pages/5127428
607,Zearaja maugeana,Endangered,open,https://biodiversity.org.au/afd/taxa/0d9d194f-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions

,scientificName,status,bionet_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
0,Acacia pataczekii,Rare,open,https://id.biodiversity.org.au/node/apni/2913609,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,pataczekii,NaN,2022-04-06T23:48:02Z,species,http://www.catalogueoflife.org/annual-checklis...
1,Acacia siculiformis,Rare,open,https://id.biodiversity.org.au/node/apni/2888960,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,siculiformis,NaN,2022-04-07T00:40:40Z,species,http://www.catalogueoflife.org/annual-checklis...
2,Acacia ulicifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2907350,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,ulicifolia,NaN,2022-06-18T06:33:18Z,species,http://www.catalogueoflife.org/annual-checklis...
3,Acacia uncifolia,Rare,open,https://id.biodiversity.org.au/node/apni/2916270,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Acacia,uncifolia,NaN,2022-06-10T10:52:23Z,species,http://eol.org/pages/26965172
4,Acanthiza pusilla archibaldi,Endangered,open,ALA_DR654_159,NaN,NaN,NaN,NaN,NaN,NaN,...,Chordata,Aves,Passeriformes,Acanthizidae,Acanthiza,pusilla,archibaldi,2018-12-19T06:16:34Z,subspecies,http://www.birds.cornell.edu/clementschecklist...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,Xanthorrhoea arenaria,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2918552,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Asphodelaceae,Xanthorrhoea,arenaria,NaN,2019-02-15T07:38:04Z,species,http://www.ubio.org/browser/details.php?nameba...
604,Xanthorrhoea bracteata,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2911139,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Liliopsida,Asparagales,Asphodelaceae,Xanthorrhoea,bracteata,NaN,2022-08-25T07:03:45Z,species,http://powo.science.kew.org/
605,Xerochrysum bicolor,Rare,open,https://id.biodiversity.org.au/node/apni/2902500,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Asterales,Asteraceae,Xerochrysum,bicolor,NaN,2020-02-19T08:20:14Z,species,http://www.catalogueoflife.org/annual-checklis...
606,Xerochrysum palustre,Vulnerable,open,https://id.biodiversity.org.au/node/apni/2890165,NaN,NaN,NaN,NaN,NaN,NaN,...,Tracheophyta,Magnoliopsida,Asterales,Asteraceae,Xerochrysum,palustre,NaN,2022-06-08T17:48:57Z,species,http://eol.org/pages/5127428


In [97]:
# there's no status but there is a matching inat taxon (id is the taxon id)
additions = pd.DataFrame(noinatstatus[noinatstatus['id'].notna()])
additions.sort_values(['scientificName'])
additions['action'] = 'ADD'
additions = additions[['action','scientificName','status_id','id','new_status','new_iucn_equivalent','new_authority','new_url','new_geoprivacy','new_place_id','new_username','new_description']]
additions.columns = additions.columns.str.replace("new_", "", regex=True)
additions = additions.rename(columns={'scientificName':'taxon_name',
                                      'id':'taxon_id',
                                  'status_id':'id'})
additions

,action,taxon_name,id,taxon_id,status,iucn_equivalent,authority,url,geoprivacy,place_id,username,description
0,ADD,Acacia pataczekii,NaN,1254534,Rare,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2913609,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
1,ADD,Acacia siculiformis,NaN,567724,Rare,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2888960,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
2,ADD,Acacia ulicifolia,NaN,139880,Rare,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2907350,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
3,ADD,Acacia uncifolia,NaN,560392,Rare,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2916270,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
4,ADD,Acanthiza pusilla archibaldi,NaN,629326,Endangered,Vulnerable,Threatened Species Protection Act 1995,ALA_DR654_159,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
...,...,...,...,...,...,...,...,...,...,...,...,...
603,ADD,Xanthorrhoea arenaria,NaN,125126,Vulnerable,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2918552,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
604,ADD,Xanthorrhoea bracteata,NaN,895045,Vulnerable,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2911139,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
605,ADD,Xerochrysum bicolor,NaN,147469,Rare,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2902500,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...
606,ADD,Xerochrysum palustre,NaN,634811,Vulnerable,Vulnerable,Threatened Species Protection Act 1995,https://id.biodiversity.org.au/node/apni/2890165,obscured,6829,peggydnew,Listed as Threatened - refer to https://www.th...


In [98]:
# write these to the file
pd.concat([updates,additions]).to_csv(sourcedir + "tas.csv", index=False)

In [99]:
# what didnt match to a taxon?
unknownToInat = noinatstatus[noinatstatus['id'].isna()]
unknownToInat

,scientificName,status,bionet_geoprivacy,lsid,status_id,taxon_id,user_id,description,iucn,authority,...,phylum,class,order,family,genus,specificEpithet,infraspecificEpithet,modified,taxonRank,references
6,Accipiter (Leucospiza) novaehollandiae,Endangered,open,https://biodiversity.org.au/afd/taxa/39c42e71-...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Agrostis australiensis,Rare,open,https://id.biodiversity.org.au/node/apni/2907094,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Agrostis diemenica,Rare,open,https://id.biodiversity.org.au/node/apni/2905826,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,Althenia australis,Presumed Extinct,open,https://id.biodiversity.org.au/taxon/apni/5129...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,Althenia marina,Rare,open,https://id.biodiversity.org.au/taxon/apni/5129...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,Xanthoparmelia mannumensis,Vulnerable,open,ALA_DR654_289,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
596,Xanthoparmelia microphyllizans,Rare,open,ALA_DR654_530,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,Xanthoparmelia vicaria,Rare,open,ALA_DR654_353,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,Xanthoparmelia vicariella,Rare,open,ALA_DR654_354,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### are there any that need to be removed?
TAS sensitive list count: 0
TAS inat statuses count: 0

updates to inat status: 0
additional inat status: 0
TAS statuses we can't find a taxon match for in iNaturalist: 0
total: 0 (explainable via the various genus/section entries that we matched to in the taxonomy)

inat statuses left over: 18-17=1 that may need checking

In [100]:
# inat statuses that aren't in added or updated
inatstatuses[~inatstatuses['taxon_id'].isin(updates['taxon_id'])]


,status_id,taxon_id,user_id,place_id,source_id,authority,status,url,description,geoprivacy,...,genus,specificEpithet,infraspecificEpithet,modified,scientificName,taxonRank,references,preferred_common_name,is_active,current_synonymous_taxon_ids
2566,160929,1092444,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Spyridium-...,,obscured,...,Spyridium,eriocephalum,eriocephalum,2021-11-09T06:11:14Z,Spyridium eriocephalum eriocephalum,variety,,,,
3353,267438,1413424,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Pneumatopt...,,obscured,...,Pakau,pennigera,,2022-09-25T23:02:37Z,Pakau pennigera,species,https://shop.brit.org/A-Generic-Classification...,,,
1251,152435,19250,708886,9994,16652,NT NRETAS,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Polytelis,alexandrae,,2019-08-27T01:09:01Z,Polytelis alexandrae,species,http://www.birdlife.org/datazone/speciesfactsh...,,,
904,160736,353835,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Blechnum-n...,,obscured,...,Doodia,aspera,,2021-02-22T18:52:06Z,Doodia aspera,species,http://www.ubio.org/browser/details.php?nameba...,,,
1249,152433,38633,708886,9994,16652,NT NRETAS,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Bellatorias,obiri,,2019-04-30T15:19:21Z,Bellatorias obiri,species,http://reptile-database.reptarium.cz/search.ph...,,,
932,160737,401378,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Blechnum%2...,,obscured,...,Doodia,caudata,,2021-02-15T17:49:20Z,Doodia caudata,species,,,,
959,160741,405825,717122,6829,,Threatened Species Protection Act 1995,Presumed extinct,https://dpipwe.tas.gov.au/Documents/Botrychium...,,obscured,...,,,,,Sceptridium australe,,,parsley fern,False,[400050]
1252,152436,40743,708886,9994,16652,NT NRETAS,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,,,,,Hipposideros stenotis,,,Narrow-eared Roundleaf Bat,False,[1431118]
1248,152432,41326,708886,9994,16652,NT NRETAS,endangered,https://lists.ala.org.au/speciesListItem/list/...,,obscured,...,Macroderma,gigas,,2019-08-27T01:58:05Z,Macroderma gigas,species,http://www.catalogueoflife.org/annual-checklis...,,,
899,160766,699302,717122,6829,,Threatened Species Protection Act 1995,Endangered,https://dpipwe.tas.gov.au/Documents/Cyathea-Xm...,,obscured,...,Cyathea,marcescens,,2022-01-05T00:50:29Z,Cyathea marcescens,species,http://eol.org/pages/44065177,,,
